<a href="https://colab.research.google.com/github/MeenTers/Project/blob/main/Project_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
from torch import nn, load, utils
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms, models
from torchvision.models.vgg import model_urls
from os import path, listdir
model_urls['vgg19'] = model_urls['vgg19'].replace('https://', 'http://')

In [3]:
from scipy.spatial import distance_matrix
from scipy.spatial import distance

In [101]:
import pandas as pd

# Prepearing the model

In [4]:
class VGG:
	def __init__(self):
		model = models.vgg19(pretrained=True, progress=True)
		model.classifier = nn.Sequential(*list(model.classifier.children())[:3])
		self.model = model.cuda().eval()

	def __call__(self, x):
		return self.model(x)

In [5]:
vgg = VGG()

Downloading: "http://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

# Features Extraction

In [6]:
def get_features(model, loader):
    features = []
    with torch.no_grad():
        for batch, _ in tqdm(loader):
            if torch.cuda.is_available():
                batch = batch.cuda()
            b_features = model(batch).detach().cpu().numpy()
            for f in b_features:
                features.append(f)

    return features

# Loading the data

In [7]:
def get_dataset(images_path):
  transform = transforms.Compose([
    transforms.Resize(size=32),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  dataset = datasets.ImageFolder(images_path, transform=transform)
  loader = utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=1, pin_memory=True)
  return dataset, loader

# Test multi-view image

In [8]:
car_dataset , car_loader = get_dataset(r'/content/drive/MyDrive/Project_si/car_model/')

In [9]:
damage_dataset , damage_loader = get_dataset(r'/content/drive/MyDrive/Project_si/model_damage/')

In [10]:
car_base = get_features(model=vgg,loader=car_loader)

100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


In [11]:
len(car_base[0])

4096

In [12]:
car_base[0]

array([0.      , 0.      , 3.960284, ..., 0.      , 0.      , 0.      ],
      dtype=float32)

In [13]:
car_damage = get_features(model=vgg,loader=damage_loader)

100%|██████████| 1/1 [00:01<00:00,  1.81s/it]


In [14]:
len(car_damage[0])

4096

# Distance

In [15]:
car_image = '/content/drive/MyDrive/Project_si/car_model/ID1'
damage_image = '/content/drive/MyDrive/Project_si/model_damage/ID1'

In [16]:
distance.euclidean(car_base[1], car_damage[1])

13.258048057556152

# compute distance


In [117]:
def myeuclidean(basecar,diff_view):
    A = []
    for i in range(len(diff_view)):
        x = distance.euclidean(basecar[0],diff_view[i])
        A.append(x)
    return(A)

# ID1

In [17]:
base_car_dataset, base_car_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID1') 
view_car_dataset, view_car_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID1')

In [18]:
base_car = get_features(vgg,base_car_loader)
view_car = get_features(vgg,view_car_loader)

100%|██████████| 1/1 [00:04<00:00,  4.11s/it]


In [119]:
list_A = myeuclidean(base_car,view_car)

In [120]:
df = pd.DataFrame(list_A,columns=['Distance'])
df

,Distance
0,39.948906
1,37.413815
2,44.718765
3,49.927807
4,42.918880
5,48.434917
6,43.104614


# ID2

In [59]:
base_car1_dataset, base_car1_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID2') 
view_car1_dataset, view_car1_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID2')

In [27]:
base_car1 = get_features(vgg,base_car1_loader)
view_car1 = get_features(vgg,view_car1_loader)

100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


In [99]:
myeuclidean(base_car1,view_car1)

[9.628503799438477, 23.583209991455078, 30.316486358642578, 28.636341094970703]


# ID3


In [32]:
base_car2_dataset, base_car2_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/Base_car/ID3') 
view_car2_dataset, view_car2_loader = get_dataset('/content/drive/MyDrive/Project_si/Test_Dist/different_view/ID3')

In [33]:
base_car2 = get_features(vgg,base_car2_loader)
view_car2 = get_features(vgg,view_car2_loader)

100%|██████████| 1/1 [00:02<00:00,  2.48s/it]


In [100]:
myeuclidean(base_car2,view_car2)

[43.40465545654297, 62.07086944580078, 57.99443817138672, 83.56273651123047]
